In [735]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

I am reloading and merging these datasets. I realized some mistakes I made in the last section. I edited the data wrangling section to fix some of the mistakes.

In [737]:
fred = pd.read_csv('fred_tot.csv')
weather = pd.read_csv('weather.csv')

In [738]:
fred.head(10)

,observation_date,GDPC1,ICSA,Inflation,UNRATE,SP500,DRCCLACBS,DTWEXBGS,FEDFUNDS,IMPGS,INDPRO,PERMIT,TCU,VIXCLS
0,2015-10-01,18892.206,272000.0,0.118627,5.0,1923.82,2.16,111.0439,0.12,2745.517,100.1563,1160.0,76.752,22.55
1,2015-10-02,18892.206,272000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,20.94
2,2015-10-03,18892.206,267000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,NaN
3,2015-10-04,18892.206,267000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,NaN
4,2015-10-05,18892.206,267000.0,0.118627,5.0,1987.05,2.16,110.4445,0.12,2745.517,100.1563,1160.0,76.752,19.54
5,2015-10-06,18892.206,267000.0,0.118627,5.0,1979.92,2.16,110.1371,0.12,2745.517,100.1563,1160.0,76.752,19.40
6,2015-10-07,18892.206,267000.0,0.118627,5.0,1995.83,2.16,109.8162,0.12,2745.517,100.1563,1160.0,76.752,18.40
7,2015-10-08,18892.206,267000.0,0.118627,5.0,2013.43,2.16,109.6452,0.12,2745.517,100.1563,1160.0,76.752,17.42
8,2015-10-09,18892.206,267000.0,0.118627,5.0,2014.89,2.16,109.0782,0.12,2745.517,100.1563,1160.0,76.752,17.08
9,2015-10-10,18892.206,265000.0,0.118627,5.0,2014.89,2.16,109.0782,0.12,2745.517,100.1563,1160.0,76.752,NaN


In [739]:
fred.set_index('observation_date', inplace=True)

In [740]:
weather.head()

,observation_date,State,Event Type,Property Damage,Total Injuries and Deaths
0,1950-04-28,OKLAHOMA,Tornado,250K,0
1,1950-04-29,TEXAS,Tornado,25K,0
2,1950-07-05,PENNSYLVANIA,Tornado,25K,2
3,1950-07-05,PENNSYLVANIA,Tornado,2.5K,0
4,1950-07-24,PENNSYLVANIA,Tornado,2.5K,0


In [741]:
weather.set_index('observation_date', inplace=True)

Function that removes the letters from the property damage column and converts it to its correct number. 

In [743]:
def convert_letter(val):
    if isinstance(val, str): 
        val = val.strip().lower()
        try:
            if len(val) == 0: 
                return 0
            elif val == 'k':
                return 1_000
            elif val == 'm':
                return 1_000_000
            elif val == 'b':
                return 1_000_000_000
            elif val.endswith('k'):
                return int(float(val[:-1]) * 1_000)
            elif val.endswith('m'):
                return int(float(val[:-1]) * 1_000_000)
            elif val.endswith('b'):
                return int(float(val[:-1]) * 1_000_000_000)
            else:
                return float(val)
        except ValueError:
            return np.nan 
    elif isinstance(val, (int, float)):
        return val
    elif val is None:
        return np.nan
    else:
        return np.nan

In [744]:
weather['Property Damage'] = weather['Property Damage'].apply(convert_letter)

I don't want a row for each weather event so here we are aggregating the weather columns. 

In [746]:
daily_summary = weather.groupby('observation_date').agg({
    'Property Damage': 'sum',
    'Total Injuries and Deaths': 'sum',
    'Event Type': 'count'
}).rename(columns={
    'Event Type': 'num_events'
})

In [747]:
daily_summary.head()

,Property Damage,Total Injuries and Deaths,num_events
observation_date,,,
1950-01-03,3025000.0,7,4
1950-01-13,2500.0,2,1
1950-01-25,500000.0,5,2
1950-01-26,0.0,2,1
1950-02-11,550000.0,24,4


In [748]:
df = pd.merge(fred, daily_summary, on='observation_date', how='outer')
df.head()

,GDPC1,ICSA,Inflation,UNRATE,SP500,DRCCLACBS,DTWEXBGS,FEDFUNDS,IMPGS,INDPRO,PERMIT,TCU,VIXCLS,Property Damage,Total Injuries and Deaths,num_events
observation_date,,,,,,,,,,,,,,,,
1950-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3025000.0,7.0,4.0
1950-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2500.0,2.0,1.0
1950-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500000.0,5.0,2.0
1950-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,1.0
1950-02-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,550000.0,24.0,4.0


In [749]:
print(df.index.dtype)

object


We need a continuous time series (no gaps) so here we are adding rows for missing days. This will be the next several blocks. 

In [751]:
full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
print(full_range.dtype)
print(df.index[:5])
print(full_range[:5])

datetime64[ns]
Index(['1950-01-03', '1950-01-13', '1950-01-25', '1950-01-26', '1950-02-11'], dtype='object', name='observation_date')
DatetimeIndex(['1950-01-03', '1950-01-04', '1950-01-05', '1950-01-06',
               '1950-01-07'],
              dtype='datetime64[ns]', freq='D')


In [752]:
df.index = pd.to_datetime(df.index)

In [753]:
print(df.index.dtype)

datetime64[ns]


There was a huge gap in the data from 2015-2016, I dropped the singlular 2015 row. 

In [755]:
df = df[df.index >= '2015-10-01']

In [756]:
df.head()

,GDPC1,ICSA,Inflation,UNRATE,SP500,DRCCLACBS,DTWEXBGS,FEDFUNDS,IMPGS,INDPRO,PERMIT,TCU,VIXCLS,Property Damage,Total Injuries and Deaths,num_events
observation_date,,,,,,,,,,,,,,,,
2015-10-01,18892.206,272000.0,0.118627,5.0,1923.82,2.16,111.0439,0.12,2745.517,100.1563,1160.0,76.752,22.55,11161550.0,2.0,190.0
2015-10-02,18892.206,272000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,20.94,25114500.0,3.0,112.0
2015-10-03,18892.206,267000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,NaN,51188540.0,3.0,114.0
2015-10-04,18892.206,267000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,NaN,88252280.0,35.0,135.0
2015-10-05,18892.206,267000.0,0.118627,5.0,1987.05,2.16,110.4445,0.12,2745.517,100.1563,1160.0,76.752,19.54,3582000.0,3.0,39.0


In [757]:
full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
df = df.reindex(full_range)
df.head()

,GDPC1,ICSA,Inflation,UNRATE,SP500,DRCCLACBS,DTWEXBGS,FEDFUNDS,IMPGS,INDPRO,PERMIT,TCU,VIXCLS,Property Damage,Total Injuries and Deaths,num_events
2015-10-01,18892.206,272000.0,0.118627,5.0,1923.82,2.16,111.0439,0.12,2745.517,100.1563,1160.0,76.752,22.55,11161550.0,2.0,190.0
2015-10-02,18892.206,272000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,20.94,25114500.0,3.0,112.0
2015-10-03,18892.206,267000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,NaN,51188540.0,3.0,114.0
2015-10-04,18892.206,267000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,NaN,88252280.0,35.0,135.0
2015-10-05,18892.206,267000.0,0.118627,5.0,1987.05,2.16,110.4445,0.12,2745.517,100.1563,1160.0,76.752,19.54,3582000.0,3.0,39.0


Here we are creating a column that flags significant property damage. 

In [759]:
df['flagged_event'] = ((df['Property Damage'] > 100000) & (df['num_events'] >= 1)).astype(int)

In [760]:
df.head()

,GDPC1,ICSA,Inflation,UNRATE,SP500,DRCCLACBS,DTWEXBGS,FEDFUNDS,IMPGS,INDPRO,PERMIT,TCU,VIXCLS,Property Damage,Total Injuries and Deaths,num_events,flagged_event
2015-10-01,18892.206,272000.0,0.118627,5.0,1923.82,2.16,111.0439,0.12,2745.517,100.1563,1160.0,76.752,22.55,11161550.0,2.0,190.0,1
2015-10-02,18892.206,272000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,20.94,25114500.0,3.0,112.0,1
2015-10-03,18892.206,267000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,NaN,51188540.0,3.0,114.0,1
2015-10-04,18892.206,267000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,NaN,88252280.0,35.0,135.0,1
2015-10-05,18892.206,267000.0,0.118627,5.0,1987.05,2.16,110.4445,0.12,2745.517,100.1563,1160.0,76.752,19.54,3582000.0,3.0,39.0,1


In [761]:
df.isnull().sum()

GDPC1                         570
ICSA                          570
Inflation                     570
UNRATE                        570
SP500                           0
DRCCLACBS                     204
DTWEXBGS                        6
FEDFUNDS                       53
IMPGS                         204
INDPRO                         53
PERMIT                         53
TCU                            53
VIXCLS                       1024
Property Damage              1396
Total Injuries and Deaths    1396
num_events                   1396
flagged_event                   0
dtype: int64

Forward filling the market index data. 

In [763]:
df[['GDPC1', 'ICSA', 'Inflation', 'UNRATE', 'SP500', 'DRCCLACBS', 
                      'DTWEXBGS', 'FEDFUNDS', 'IMPGS', 'INDPRO', 'PERMIT', 'TCU', 'VIXCLS']]= df[['GDPC1', 'ICSA', 'Inflation', 'UNRATE', 'SP500', 'DRCCLACBS', 
                      'DTWEXBGS', 'FEDFUNDS', 'IMPGS', 'INDPRO', 'PERMIT', 'TCU', 'VIXCLS']].ffill()


In [764]:
df.isnull().sum()

GDPC1                           0
ICSA                            0
Inflation                       0
UNRATE                          0
SP500                           0
DRCCLACBS                       0
DTWEXBGS                        0
FEDFUNDS                        0
IMPGS                           0
INDPRO                          0
PERMIT                          0
TCU                             0
VIXCLS                          0
Property Damage              1396
Total Injuries and Deaths    1396
num_events                   1396
flagged_event                   0
dtype: int64

We can't forward fill the weather data. This would make it look like there was weather events on days where there wasn't any. Instead, we are filling them with 0. 

In [766]:
df[['Property Damage', 'Total Injuries and Deaths', 'num_events']] = df[['Property Damage', 'Total Injuries and Deaths', 'num_events']].fillna(0)


In [767]:
df.isnull().sum()

GDPC1                        0
ICSA                         0
Inflation                    0
UNRATE                       0
SP500                        0
DRCCLACBS                    0
DTWEXBGS                     0
FEDFUNDS                     0
IMPGS                        0
INDPRO                       0
PERMIT                       0
TCU                          0
VIXCLS                       0
Property Damage              0
Total Injuries and Deaths    0
num_events                   0
flagged_event                0
dtype: int64

Our target variable for the model is going to be the percent change in the market indexes. First we calculate the percent change for each column across the rows. Next we will take the mean of each row. This will be the 'Percent Market Change' for the day. 

In [769]:
market_change = df[['GDPC1', 'ICSA', 'Inflation', 'UNRATE', 'SP500', 'DRCCLACBS', 
                      'DTWEXBGS', 'FEDFUNDS', 'IMPGS', 'INDPRO', 'PERMIT', 'TCU', 'VIXCLS']].pct_change()
market_change.head(20)

,GDPC1,ICSA,Inflation,UNRATE,SP500,DRCCLACBS,DTWEXBGS,FEDFUNDS,IMPGS,INDPRO,PERMIT,TCU,VIXCLS
2015-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-10-02,0.0,0.000000,0.0,0.0,0.014315,0.0,-0.003092,0.0,0.0,0.0,0.0,0.0,-0.071397
2015-10-03,0.0,-0.018382,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2015-10-04,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2015-10-05,0.0,0.000000,0.0,0.0,0.018290,0.0,-0.002313,0.0,0.0,0.0,0.0,0.0,-0.066858
2015-10-06,0.0,0.000000,0.0,0.0,-0.003588,0.0,-0.002783,0.0,0.0,0.0,0.0,0.0,-0.007165
2015-10-07,0.0,0.000000,0.0,0.0,0.008036,0.0,-0.002914,0.0,0.0,0.0,0.0,0.0,-0.051546
2015-10-08,0.0,0.000000,0.0,0.0,0.008818,0.0,-0.001557,0.0,0.0,0.0,0.0,0.0,-0.053261
2015-10-09,0.0,0.000000,0.0,0.0,0.000725,0.0,-0.005171,0.0,0.0,0.0,0.0,0.0,-0.019518
2015-10-10,0.0,-0.007491,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [770]:
def rename_col(suffix): 
     market_change.rename(columns={col: f"{col}_{suffix}" for col in market_change}, inplace=True)

rename_col('pct')
market_change.head()
        

,GDPC1_pct,ICSA_pct,Inflation_pct,UNRATE_pct,SP500_pct,DRCCLACBS_pct,DTWEXBGS_pct,FEDFUNDS_pct,IMPGS_pct,INDPRO_pct,PERMIT_pct,TCU_pct,VIXCLS_pct
2015-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-10-02,0.0,0.000000,0.0,0.0,0.014315,0.0,-0.003092,0.0,0.0,0.0,0.0,0.0,-0.071397
2015-10-03,0.0,-0.018382,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2015-10-04,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2015-10-05,0.0,0.000000,0.0,0.0,0.018290,0.0,-0.002313,0.0,0.0,0.0,0.0,0.0,-0.066858


In [771]:
market_change['combined_pct_change'] = market_change.mean(axis=1)
market_change.head()

,GDPC1_pct,ICSA_pct,Inflation_pct,UNRATE_pct,SP500_pct,DRCCLACBS_pct,DTWEXBGS_pct,FEDFUNDS_pct,IMPGS_pct,INDPRO_pct,PERMIT_pct,TCU_pct,VIXCLS_pct,combined_pct_change
2015-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-10-02,0.0,0.000000,0.0,0.0,0.014315,0.0,-0.003092,0.0,0.0,0.0,0.0,0.0,-0.071397,-0.004629
2015-10-03,0.0,-0.018382,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,-0.001414
2015-10-04,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2015-10-05,0.0,0.000000,0.0,0.0,0.018290,0.0,-0.002313,0.0,0.0,0.0,0.0,0.0,-0.066858,-0.003914


In [772]:
daily_market = market_change[['combined_pct_change', 'TCU_pct']]
daily_market.head()

,combined_pct_change,TCU_pct
2015-10-01,NaN,NaN
2015-10-02,-0.004629,0.0
2015-10-03,-0.001414,0.0
2015-10-04,0.000000,0.0
2015-10-05,-0.003914,0.0


In [773]:
daily_market.index.name ='observation_date'

In [774]:
daily_market.head()

,combined_pct_change,TCU_pct
observation_date,,
2015-10-01,NaN,NaN
2015-10-02,-0.004629,0.0
2015-10-03,-0.001414,0.0
2015-10-04,0.000000,0.0
2015-10-05,-0.003914,0.0


In [775]:
df2 = pd.merge(df, daily_market, on='observation_date', how='outer')
df2.head()

,GDPC1,ICSA,Inflation,UNRATE,SP500,DRCCLACBS,DTWEXBGS,FEDFUNDS,IMPGS,INDPRO,PERMIT,TCU,VIXCLS,Property Damage,Total Injuries and Deaths,num_events,flagged_event,combined_pct_change,TCU_pct
observation_date,,,,,,,,,,,,,,,,,,,
2015-10-01,18892.206,272000.0,0.118627,5.0,1923.82,2.16,111.0439,0.12,2745.517,100.1563,1160.0,76.752,22.55,11161550.0,2.0,190.0,1,NaN,NaN
2015-10-02,18892.206,272000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,20.94,25114500.0,3.0,112.0,1,-0.004629,0.0
2015-10-03,18892.206,267000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,20.94,51188540.0,3.0,114.0,1,-0.001414,0.0
2015-10-04,18892.206,267000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,20.94,88252280.0,35.0,135.0,1,0.000000,0.0
2015-10-05,18892.206,267000.0,0.118627,5.0,1987.05,2.16,110.4445,0.12,2745.517,100.1563,1160.0,76.752,19.54,3582000.0,3.0,39.0,1,-0.003914,0.0


In [776]:
df2.drop('TCU_pct', axis=1, inplace=True)

In [777]:
df2.rename(columns={'combined_pct_change':'Percent Market Change'}, inplace=True)
df2.head()

,GDPC1,ICSA,Inflation,UNRATE,SP500,DRCCLACBS,DTWEXBGS,FEDFUNDS,IMPGS,INDPRO,PERMIT,TCU,VIXCLS,Property Damage,Total Injuries and Deaths,num_events,flagged_event,Percent Market Change
observation_date,,,,,,,,,,,,,,,,,,
2015-10-01,18892.206,272000.0,0.118627,5.0,1923.82,2.16,111.0439,0.12,2745.517,100.1563,1160.0,76.752,22.55,11161550.0,2.0,190.0,1,NaN
2015-10-02,18892.206,272000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,20.94,25114500.0,3.0,112.0,1,-0.004629
2015-10-03,18892.206,267000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,20.94,51188540.0,3.0,114.0,1,-0.001414
2015-10-04,18892.206,267000.0,0.118627,5.0,1951.36,2.16,110.7006,0.12,2745.517,100.1563,1160.0,76.752,20.94,88252280.0,35.0,135.0,1,0.000000
2015-10-05,18892.206,267000.0,0.118627,5.0,1987.05,2.16,110.4445,0.12,2745.517,100.1563,1160.0,76.752,19.54,3582000.0,3.0,39.0,1,-0.003914


Now it's time to standardize the some of the numeric features. We don't need to standardize categorical ones (like 'flagged_event') and percent change. 

In [779]:
df2['num_events_log'] = np.log1p(df2['num_events'])  #some event counts are high, some are zero. This is to make the large counts not as impactful. 
numeric_cols = df2.drop(['num_events','flagged_event', 'num_events_log', 'Percent Market Change'], axis=1).columns
scaler = StandardScaler()
df3 = df2.copy()
df3[numeric_cols] = scaler.fit_transform(df3[numeric_cols])
df3[numeric_cols].head()

,GDPC1,ICSA,Inflation,UNRATE,SP500,DRCCLACBS,DTWEXBGS,FEDFUNDS,IMPGS,INDPRO,PERMIT,TCU,VIXCLS,Property Damage,Total Injuries and Deaths
observation_date,,,,,,,,,,,,,,,
2015-10-01,-1.56785,-0.166212,-1.475788,0.260351,-1.426880,-0.620173,-1.159501,-1.01546,-1.139152,-0.239281,-1.512925,-0.330651,0.562201,-0.049257,-0.088246
2015-10-02,-1.56785,-0.166212,-1.475788,0.260351,-1.403413,-0.620173,-1.227661,-1.01546,-1.139152,-0.239281,-1.512925,-0.330651,0.341794,-0.031773,-0.036880
2015-10-03,-1.56785,-0.175220,-1.475788,0.260351,-1.403413,-0.620173,-1.227661,-1.01546,-1.139152,-0.239281,-1.512925,-0.330651,0.341794,0.000899,-0.036880
2015-10-04,-1.56785,-0.175220,-1.475788,0.260351,-1.403413,-0.620173,-1.227661,-1.01546,-1.139152,-0.239281,-1.512925,-0.330651,0.341794,0.047341,1.606822
2015-10-05,-1.56785,-0.175220,-1.475788,0.260351,-1.373002,-0.620173,-1.278508,-1.01546,-1.139152,-0.239281,-1.512925,-0.330651,0.150136,-0.058754,-0.036880


In [780]:
df3.drop('num_events', axis=1, inplace=True) 
df3['Percent Market Change'] = df3['Percent Market Change'].fillna(0)
df3.head(3)

,GDPC1,ICSA,Inflation,UNRATE,SP500,DRCCLACBS,DTWEXBGS,FEDFUNDS,IMPGS,INDPRO,PERMIT,TCU,VIXCLS,Property Damage,Total Injuries and Deaths,flagged_event,Percent Market Change,num_events_log
observation_date,,,,,,,,,,,,,,,,,,
2015-10-01,-1.56785,-0.166212,-1.475788,0.260351,-1.426880,-0.620173,-1.159501,-1.01546,-1.139152,-0.239281,-1.512925,-0.330651,0.562201,-0.049257,-0.088246,1,0.000000,5.252273
2015-10-02,-1.56785,-0.166212,-1.475788,0.260351,-1.403413,-0.620173,-1.227661,-1.01546,-1.139152,-0.239281,-1.512925,-0.330651,0.341794,-0.031773,-0.036880,1,-0.004629,4.727388
2015-10-03,-1.56785,-0.175220,-1.475788,0.260351,-1.403413,-0.620173,-1.227661,-1.01546,-1.139152,-0.239281,-1.512925,-0.330651,0.341794,0.000899,-0.036880,1,-0.001414,4.744932


Creating lag features for our target variable. In the modeling step we need data from previous days. 

In [782]:
for lag in range(1, 8):
    df3[f'lag_{lag}'] = df3['Percent Market Change'].shift(lag)

In [783]:
df3.head(8)

,GDPC1,ICSA,Inflation,UNRATE,SP500,DRCCLACBS,DTWEXBGS,FEDFUNDS,IMPGS,INDPRO,...,flagged_event,Percent Market Change,num_events_log,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
observation_date,,,,,,,,,,,,,,,,,,,,,
2015-10-01,-1.56785,-0.166212,-1.475788,0.260351,-1.426880,-0.620173,-1.159501,-1.01546,-1.139152,-0.239281,...,1,0.000000,5.252273,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-10-02,-1.56785,-0.166212,-1.475788,0.260351,-1.403413,-0.620173,-1.227661,-1.01546,-1.139152,-0.239281,...,1,-0.004629,4.727388,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2015-10-03,-1.56785,-0.175220,-1.475788,0.260351,-1.403413,-0.620173,-1.227661,-1.01546,-1.139152,-0.239281,...,1,-0.001414,4.744932,-0.004629,0.000000,NaN,NaN,NaN,NaN,NaN
2015-10-04,-1.56785,-0.175220,-1.475788,0.260351,-1.403413,-0.620173,-1.227661,-1.01546,-1.139152,-0.239281,...,1,0.000000,4.912655,-0.001414,-0.004629,0.000000,NaN,NaN,NaN,NaN
2015-10-05,-1.56785,-0.175220,-1.475788,0.260351,-1.373002,-0.620173,-1.278508,-1.01546,-1.139152,-0.239281,...,1,-0.003914,3.688879,0.000000,-0.001414,-0.004629,0.000000,NaN,NaN,NaN
2015-10-06,-1.56785,-0.175220,-1.475788,0.260351,-1.379077,-0.620173,-1.339540,-1.01546,-1.139152,-0.239281,...,0,-0.001041,3.828641,-0.003914,0.000000,-0.001414,-0.004629,0.000000,NaN,NaN
2015-10-07,-1.56785,-0.175220,-1.475788,0.260351,-1.365520,-0.620173,-1.403253,-1.01546,-1.139152,-0.239281,...,1,-0.003571,2.890372,-0.001041,-0.003914,0.000000,-0.001414,-0.004629,0.000000,NaN
2015-10-08,-1.56785,-0.175220,-1.475788,0.260351,-1.350523,-0.620173,-1.437204,-1.01546,-1.139152,-0.239281,...,1,-0.003538,2.639057,-0.003571,-0.001041,-0.003914,0.000000,-0.001414,-0.004629,0.0


In [784]:
df3 = df3[df3.index >= '2015-10-09']
df3.head()

,GDPC1,ICSA,Inflation,UNRATE,SP500,DRCCLACBS,DTWEXBGS,FEDFUNDS,IMPGS,INDPRO,...,flagged_event,Percent Market Change,num_events_log,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
observation_date,,,,,,,,,,,,,,,,,,,,,
2015-10-09,-1.56785,-0.175220,-1.475788,0.260351,-1.349279,-0.620173,-1.549778,-1.01546,-1.139152,-0.239281,...,0,-0.001843,3.663562,-0.003538,-0.003571,-0.001041,-0.003914,0.000000,-0.001414,-0.004629
2015-10-10,-1.56785,-0.178823,-1.475788,0.260351,-1.349279,-0.620173,-1.549778,-1.01546,-1.139152,-0.239281,...,1,-0.000576,3.737670,-0.001843,-0.003538,-0.003571,-0.001041,-0.003914,0.000000,-0.001414
2015-10-11,-1.56785,-0.178823,-1.475788,0.260351,-1.349279,-0.620173,-1.549778,-1.01546,-1.139152,-0.239281,...,1,0.000000,4.875197,-0.000576,-0.001843,-0.003538,-0.003571,-0.001041,-0.003914,0.000000
2015-10-12,-1.56785,-0.178823,-1.475788,0.260351,-1.347089,-0.620173,-1.549778,-1.01546,-1.139152,-0.239281,...,0,-0.004000,3.258097,0.000000,-0.000576,-0.001843,-0.003538,-0.003571,-0.001041,-0.003914
2015-10-13,-1.56785,-0.178823,-1.475788,0.260351,-1.358823,-0.620173,-1.483524,-1.01546,-1.139152,-0.239281,...,1,0.006846,3.178054,-0.004000,0.000000,-0.000576,-0.001843,-0.003538,-0.003571,-0.001041


We will be using a 80/20 training/test split. 

In [786]:
df3 = df3.sort_index()
train_size = int(len(df3) * 0.8)
X = df3.drop('Percent Market Change', axis =1) 
y = df3['Percent Market Change']
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]